In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import risk_models
from sklearn import preprocessing
from pypfopt.efficient_frontier import EfficientFrontier
from datetime import date, datetime, timedelta
from arch import arch_model
from pypfopt import expected_returns
import tensorflow as tf
from tqdm import tqdm

In [2]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Close'][i+dif])-np.log(Database['Close'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))

#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index) 
    return Data.dropna()

In [13]:
# start = '2009-01-01';end = '2013-01-01'
start='2008-01-01'; end='2015-12-31'; 
asset = "^GSPC"
Lag=1; LagSD=4
IndexEndDays=yf.download(asset,start=start,  end=end, progress=False).resample('W-FRI').last().index
Database=yf.download(asset,start, end, progress=False).resample('W-FRI').last()
Data = DatabaseGeneration(Database, Lag, LagSD)
Data



,DailyReturns,SD,TrueSD,DailyReturnsOld
Date,,,,
2008-02-08,-0.047047,0.042420,0.026639,0.047558
2008-02-15,0.013949,0.048085,0.018960,-0.047047
2008-02-22,0.002308,0.039160,0.013653,0.013949
2008-02-29,-0.016753,0.039178,0.025985,0.002308
2008-03-07,-0.028401,0.026639,0.025200,-0.016753
...,...,...,...,...
2015-11-13,-0.036955,0.007619,0.028272,0.009496
2015-11-20,0.032165,0.024985,0.029000,-0.036955
2015-11-27,0.000450,0.028771,0.019059,0.032165


In [4]:
from arch.__future__ import reindexing

In [5]:
#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=4).variance.dropna())
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11

def TARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    TARCH11 = arch_model(AR_Data, p=1, o=1, q=1, power=1.0, dist ='t')
    res_TARCH11 = TARCH11.fit(disp='off')
    CV_TARCH11 = res_TARCH11.conditional_volatility
    For_CV_TARCH11 = []
    for i in range(4):
        forecast = res_TARCH11.forecast(start=AR_Data.shape[0]-1, horizon=1)
        For_CV_TARCH11.append(forecast.variance.iloc[-1,:].values[0])
        AR_Data = np.append(AR_Data, forecast.mean.iloc[-1,:].values[0])
        TARCH11 = arch_model(AR_Data, p=1, o=1, q=1, power=1.0, dist ='t')
        res_TARCH11 = TARCH11.fit(disp='off')
    return TARCH11, res_TARCH11, CV_TARCH11, np.array(For_CV_TARCH11)



In [6]:
GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11 = TARCH_Model_Student (Data)


In [7]:
For_CV_GARCH11

array([2.87712141, 1.96441166, 1.43879885, 1.13034033])

In [8]:
CV_GARCH11

Date
2009-02-06    3.626604
2009-02-13    3.246392
2009-02-20    4.666402
2009-02-27    6.495362
2009-03-06    6.549102
                ...   
2012-11-16    2.779206
2012-11-23    2.867859
2012-11-30    2.577729
2012-12-07    1.985294
2012-12-14    1.685259
Freq: W-FRI, Name: cond_vol, Length: 202, dtype: float64

# Step 1, modify GARCH models to be multistep

If you find that the TARCH model does not support a horizon greater than 1, one workaround could be to implement recursive forecasting manually. This would involve using the model to make a one-step ahead forecast, appending that forecast to your time series, and then making the next one-step ahead forecast, and so on until you have made 4 forecasts. However, this approach would also be based on the assumption that future residuals are zero, and it would be computationally more intensive.

In [48]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Close'][i+dif])-np.log(Database['Close'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))


#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index) 
    return Data.dropna()

#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=4).variance.dropna())
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11

#Fitting of GJR_GARCH(1,1)
def GJR_GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GJR_GARCH11 = arch_model(AR_Data, p=1, o=1, q=1, dist ='t')
    res_GJR_GARCH11 = GJR_GARCH11.fit(disp='off')
    CV_GJR_GARCH11 = res_GJR_GARCH11.conditional_volatility
    For_CV_GJR_GARCH11 = np.array(res_GJR_GARCH11.forecast(horizon=4).variance.dropna())
    return GJR_GARCH11, res_GJR_GARCH11, CV_GJR_GARCH11, For_CV_GJR_GARCH11

#Fitting of TARCH(1,1)
def TARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    TARCH11 = arch_model(AR_Data, p=1, o=1, q=1, power=1.0, dist ='t')
    res_TARCH11 = TARCH11.fit(disp='off')
    CV_TARCH11 = res_TARCH11.conditional_volatility
    For_CV_TARCH11 = np.array(res_TARCH11.forecast(horizon=4,method= "bootstrap").variance.dropna())
    return TARCH11, res_TARCH11, CV_TARCH11, For_CV_TARCH11

#Fitting of EGARCH(1,1)
def EGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    EGARCH11 = arch_model(AR_Data, dist ='t', vol="EGARCH")
    res_EGARCH11 = EGARCH11.fit(disp='off')
    CV_EGARCH11 = res_EGARCH11.conditional_volatility
    For_CV_EGARCH11 = np.array(res_EGARCH11.forecast(horizon=4,method="bootstrap").variance.dropna())
    return EGARCH11, res_EGARCH11,CV_EGARCH11, For_CV_EGARCH11

#Fitting of Absolute Value GARCH(1,1)
def AVGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    AVGARCH11 = arch_model(AR_Data, dist ='t', power=1)
    res_AVGARCH11 = AVGARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_AVGARCH11 = res_AVGARCH11.conditional_volatility
    For_CV_AVGARCH11 = np.array(res_AVGARCH11.forecast(horizon=4,method="bootstrap").variance.dropna())
    return AVGARCH11, res_AVGARCH11, CV_AVGARCH11, For_CV_AVGARCH11

#Fitting of FIGARCH11(1,1)
def FIGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res_FIGARCH11 = FIGARCH11.fit(disp='off')
    CV_FIGARCH11 = res_FIGARCH11.conditional_volatility
    For_CV_FIGARCH11 = np.array(res_FIGARCH11.forecast(horizon=4,method="bootstrap").variance.dropna())
    return FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11

#this old code was inconsistent with the original, it may be been forecasting steps y2,y3,y4,y5 instead of y1,y2,y3,y4
# def Transformer_Database (Timestep, XData_AR, YData_AR):
#     Features = XData_AR.shape[1]
#     Sample = XData_AR.shape[0] - Timestep - 3  # Adjusted to allow for a 4-step-ahead target
#     XDataTrainScaledRNN = np.zeros([Sample, Timestep, Features])
#     YDataTrainRNN = np.zeros([Sample, 4])  # Adjusted for 4-step-ahead forecasts
    
#     for i in range(Sample):
#         XDataTrainScaledRNN[i,:,:] = XData_AR[i:(Timestep+i)]
#         YDataTrainRNN[i, :] = YData_AR[(Timestep+i):(Timestep+i+4)]  # 4-step-ahead target
    
#     return XDataTrainScaledRNN, YDataTrainRNN

def Transformer_Database (Timestep, XData_AR, YData_AR):
    Features = XData_AR.shape[1]
    Sample = XData_AR.shape[0] - Timestep - 2  # Adjusted to allow for a 4-step-ahead target
    XDataTrainScaledRNN = np.zeros([Sample, Timestep, Features])
    YDataTrainRNN = np.zeros([Sample, 4])  # Adjusted for 4-step-ahead forecasts
    
    for i in range(Sample):
        XDataTrainScaledRNN[i,:,:] = XData_AR[i:(Timestep+i)]
        YDataTrainRNN[i, :] = YData_AR[(Timestep+i-1):(Timestep+i+3)]  # 4-step-ahead target
    
    return XDataTrainScaledRNN, YDataTrainRNN

#MultiHeadSelfAttention
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(query, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(key, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(value, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(concat_attention)  # (batch_size, seq_len, embed_dim)
        return output
        
#Transformer Keras Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.nb_dict = {}; self.Bagging=5
        for i in range(self.Bagging):
          self.nb_dict["att{0}".format(i)]=MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    def call(self, inputs, training):
        self.att_dict = {}
        for i in range(self.Bagging):
          self.att_dict["att{0}".format(i)]=self.nb_dict["att{0}".format(i)](tf.keras.layers.Dropout(.1)(inputs))
          if i==0: 
            self.att_dict["attn_output"]=self.att_dict["att{0}".format(i)]/self.Bagging 
          else: 
            self.att_dict["attn_output"]=self.att_dict["attn_output"]+self.att_dict["att{0}".format(i)]/self.Bagging
        attn_output = self.dropout1(self.att_dict["attn_output"], training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
#Database is calculated
def DatabaseGenerationForecast (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index) 
    return Data

def Transformer_Model (Shape1, Shape2, HeadsAttention,Dropout, LearningRate):
    #Model struture is defined
    Input = tf.keras.Input(shape=(Shape1,Shape2), name="Input")
    #LSTM is applied on top of the transformer
    X = tf.keras.layers.LSTM(units=16, dropout=Dropout, return_sequences=True)(Input)
    #Tranformer architecture is implemented
    transformer_block_1 = TransformerBlock(embed_dim=16, num_heads=HeadsAttention, ff_dim=8, rate=Dropout)
    X = transformer_block_1(X)
    #Dense layers are used
    X = tf.keras.layers.GlobalAveragePooling1D()(X)
    X = tf.keras.layers.Dense(8, activation=tf.nn.sigmoid)(X)
    X = tf.keras.layers.Dropout(Dropout)(X)
    Output = tf.keras.layers.Dense(4, activation=tf.nn.sigmoid, name="Output")(X)
    model = tf.keras.Model(inputs=Input, outputs=Output)
    #Optimizer is defined
    Opt = tf.keras.optimizers.legacy.Adam(learning_rate=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    #Model is compiled
    model.compile(optimizer=Opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model



def DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH):
    Data_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).iloc[(-LagSD+1)]
    Index_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).index[(-LagSD+1)]
    XDataForecast=[]
    # Flatten the double-nested lists
    For_CV_GARCH = [item for sublist in For_CV_GARCH for item in sublist]
    For_CV_GJR_GARCH = [item for sublist in For_CV_GJR_GARCH for item in sublist]
    For_CV_TARCH = [item for sublist in For_CV_TARCH for item in sublist]
    For_CV_EGARCH = [item for sublist in For_CV_EGARCH for item in sublist]
    For_CV_AVGARCH = [item for sublist in For_CV_AVGARCH for item in sublist]
    For_CV_FIGARCH = [item for sublist in For_CV_FIGARCH for item in sublist]
    for i in range(len(For_CV_AVGARCH)):
        forecast={'SD': Data_Forecast['SD'], 'DailyReturnsOld': Data_Forecast['DailyReturnsOld'], 
               'CV_GARCH' : For_CV_GARCH[i]/100, 'CV_GJR_GARCH' : For_CV_GJR_GARCH[i]/100, 'CV_TARCH' : For_CV_TARCH[i]/100, 
               'CV_EGARCH' : For_CV_EGARCH[i]/100, 'CV_AVGARCH' : For_CV_AVGARCH[i]/100, 'CV_FIGARCH' : For_CV_FIGARCH[i]/100}
        XDataForecast.append(pd.DataFrame([forecast], index=[Index_Forecast]))
    XDataForecast = pd.concat(XDataForecast)
    return XDataForecast, Data_Forecast['DailyReturns']

def T_ANN_ARCH_Forecast (Database,Timestep, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model):
    XDataForecast, ReturnForecast = DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH)
    XDataForecast = pd.concat([XData_AR,XDataForecast])
    XDataForecastTotalScaled = Scaled_Norm.transform(XDataForecast)
    XDataForecastTotalScaled_T, Y_T = Transformer_Database(Timestep, XDataForecastTotalScaled, np.zeros(XDataForecastTotalScaled.shape[0]))
    TransformerPrediction = model.predict(XDataForecastTotalScaled_T)
    return TransformerPrediction[-1], XDataForecast.index[-1], TransformerPrediction[0:(XDataForecastTotalScaled_T.shape[0]-1)], ReturnForecast





## Step 2: Prepare the data for the Transformer model:

In the Transformer_Database function, you need to adjust the data preparation process to handle the 4-step-ahead forecast vectors from the ARCH models. This likely involves changes to how the X and Y arrays are constructed.

In [27]:
IndexEndDays=yf.download(asset,start=start,  end=end, progress=False).resample('W-FRI').last().index
i = 30
Database=yf.download(asset,start=IndexEndDays[i].date()-timedelta(days=780), end=IndexEndDays[i].date(), progress=False).resample('W-FRI').last()
#Database for fitting the models is generated
Data = DatabaseGeneration(Database, Lag, LagSD)

In [51]:
Lag=1; LagSD=5; Timestep=10; Dropout=0.05; LearningRate=0.01; Epochs = 100;BatchSize=64
GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
#Database contaning AR models is generated
Data_AR=pd.concat([Data, CV_GARCH.rename('CV_GARCH')/100, CV_GJR_GARCH.rename('CV_GJR_GARCH')/100, CV_TARCH.rename('CV_TARCH')/100, 
                    CV_EGARCH.rename('CV_EGARCH')/100, CV_AVGARCH.rename('CV_AVGARCH')/100, CV_FIGARCH.rename('CV_FIGARCH')/100], axis=1)
if Data_AR.shape[0]!=Data.shape[0]: print("Error in DB Generation")
# #Original explanatory and response variables are generated
XData_AR = Data_AR.drop(Data_AR.columns[[0,2]], axis=1);YData_AR = Data_AR['TrueSD']
# #Data is normalized
Scaled_Norm = preprocessing.StandardScaler().fit(XData_AR); XData_AR_Norm = Scaled_Norm.transform(XData_AR)
#Data for fitting the transformer model is generated
XData_AR_Norm_T, YData_AR_Norm_T= Transformer_Database(Timestep, XData_AR_Norm, YData_AR)
# #Model with transformer layer is defined
model = Transformer_Model(XData_AR_Norm_T.shape[1], XData_AR_Norm_T.shape[2], HeadsAttention=4, Dropout=Dropout, LearningRate=LearningRate) #this shifts weekly True SD is working fine here
model.fit(XData_AR_Norm_T, YData_AR_Norm_T, epochs=Epochs, verbose=0, batch_size=BatchSize); tf.keras.backend.clear_session()
# #T_ANN_ARCH_Forecast
XDataForecast, ReturnForecast = DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH)
XDataForecast = pd.concat([XData_AR,XDataForecast])
XDataForecastTotalScaled = Scaled_Norm.transform(XDataForecast)
XDataForecastTotalScaled_T, Y_T = Transformer_Database(Timestep, XDataForecastTotalScaled, np.zeros(XDataForecastTotalScaled.shape[0]))
TransformerPrediction = model.predict(XDataForecastTotalScaled_T)


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/volatility.py:2774: RuntimeWarning: overflow encountered in exp
  paths[loc, :, :] = np.exp(_lnsigma2[:, m:])


In [72]:
XData_AR_Norm_T.shape[1]

10

In [6]:
def T_ANN_ARCH_Fit (Data,Database,Lag=1, LagSD=5, Timestep=10, Dropout=0.05, LearningRate=0.01, Epochs=1000, BatchSize=64):
    GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
    GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
    TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
    EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
    AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
    FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
    #Database contaning AR models is generated
    Data_AR=pd.concat([Data, CV_GARCH.rename('CV_GARCH')/100, CV_GJR_GARCH.rename('CV_GJR_GARCH')/100, CV_TARCH.rename('CV_TARCH')/100, 
                        CV_EGARCH.rename('CV_EGARCH')/100, CV_AVGARCH.rename('CV_AVGARCH')/100, CV_FIGARCH.rename('CV_FIGARCH')/100], axis=1)
    if Data_AR.shape[0]!=Data.shape[0]: print("Error in DB Generation")
    # #Original explanatory and response variables are generated
    XData_AR = Data_AR.drop(Data_AR.columns[[0,2]], axis=1);YData_AR = Data_AR['TrueSD']
    # #Data is normalized
    Scaled_Norm = preprocessing.StandardScaler().fit(XData_AR); XData_AR_Norm = Scaled_Norm.transform(XData_AR)
    #Data for fitting the transformer model is generated
    XData_AR_Norm_T, YData_AR_Norm_T= Transformer_Database(Timestep, XData_AR_Norm, YData_AR)
    #Model with transformer layer is defined
    model = Transformer_Model(XData_AR_Norm_T.shape[1], XData_AR_Norm_T.shape[2], HeadsAttention=4, Dropout=Dropout, LearningRate=LearningRate)
    model.fit(XData_AR_Norm_T, YData_AR_Norm_T, epochs=Epochs, verbose=0, batch_size=BatchSize); tf.keras.backend.clear_session()
    Forecast, Date_Forecast, TrainPrediction, ReturnForecast = T_ANN_ARCH_Forecast (Database,Timestep, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model)
    return {'Date_Forecast':Date_Forecast,'Forecast_T_ANN_ARCH':Forecast}


In [50]:
#Index of end dates, database for validation and dataframe to collect the results are created. Model variables are defined.
Start='2008-01-01'; End='2015-12-31'; 
asset = "^GSPC"
# asset_name = re.sub('[\W\d_]+', '', asset)
IndexEndDays=yf.download(asset,start=Start,  end=End, progress=False).resample('W-FRI').last().index

Lag=1; LagSD=5; Timestep=10; Dropout=0; LearningRate=0.01; Epochs=100

DataValidation = DatabaseGeneration(yf.download(asset,start='2000-01-01', end=date.today()+timedelta(days=1), progress=False).resample('W-FRI').last(), Lag, LagSD)

ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'h1': [], 'h2': [], 'h3':[], 'h4': []})
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=yf.download(asset,start=IndexEndDays[i].date()-timedelta(days=780), end=IndexEndDays[i].date(), progress=False).resample('W-FRI').last()
    #Database for fitting the models is generated
    Data = DatabaseGeneration(Database, Lag, LagSD)
    #Fitting of Transformed ANN-ARCH model, ARCH models and forecasting of the next volatility value
    T_ANN_ARCH_Model = T_ANN_ARCH_Fit (Data,Database, Lag, LagSD, Timestep, Dropout, LearningRate, Epochs)

    
    IterResults={'Date_Forecast': T_ANN_ARCH_Model['Date_Forecast'].date(), 'h1': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'][0], 'h2': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'][1],
                 'h3': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'][2], 'h4': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'][3]}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    ResultsCollection = ResultsCollection.append(IterResults_df, ignore_index=True)


    # ResultsCollection.to_csv(f'./assets/5_MTL_GARCH_{asset_name}.csv',index=False)
    ResultsCollection.to_csv(f'./test.csv',index=False)
    print(IterResults)
    


  0%|          | 0/418 [00:00<?, ?it/s]

3/3 [==============================] - 0s 2ms/step


  0%|          | 1/418 [00:05<39:34,  5.69s/it]

{'Date_Forecast': datetime.date(2007, 12, 14), 'h1': 0.03497136, 'h2': 0.022665333, 'h3': 0.03168502, 'h4': 0.030848978}


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


...

{'Date_Forecast': datetime.date(2014, 6, 13), 'h1': 0.017373288, 'h2': 0.022494443, 'h3': 0.019588254, 'h4': 0.020759266}
3/3 [==============================] - 1s 3ms/step


 82%|████████▏ | 341/418 [36:37<07:31,  5.86s/it]

{'Date_Forecast': datetime.date(2014, 6, 20), 'h1': 0.023373585, 'h2': 0.035335504, 'h3': 0.033920605, 'h4': 0.031088598}


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/volatility.py:2774: RuntimeWarning: overflow encountered in exp
  paths[loc, :, :] = np.exp(_lnsigma2[:, m:])
 82%|████████▏ | 341/418 [36:42<08:17,  6.46s/it]


IndexError: list index out of range

In [ ]:
#STD Calculation
LagSD=5
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))

In [62]:
IndexEndDays=yf.download(asset,start=Start,  end=End, progress=False).resample('W-FRI').last().index

TrueSD_df_result=pd.DataFrame({'Date': [], 'TrueSD': []})
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    Database=yf.download(asset,start=IndexEndDays[i].date()-timedelta(days=780), end=IndexEndDays[i].date(), progress=False).resample('W-FRI').last()
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    #Database for fitting the models is generated
    TrueSD  = TrueSDCalculation(DailyReturns,LagSD)
    Data = pd.DataFrame({'Date': i, 'TrueSD': TrueSD})
    TrueSD_df = pd.DataFrame(IterResults,index =[0])
    TrueSD_df_result = TrueSD_df_result.append(TrueSD_df , ignore_index=True)
    Data = Data.set_index(Index) 
    
    

 30%|██▉       | 125/418 [00:56<02:11,  2.22it/s]


KeyboardInterrupt: 

In [74]:
XData_AR

,SD,DailyReturnsOld,CV_GARCH,CV_GJR_GARCH,CV_TARCH,CV_EGARCH,CV_AVGARCH,CV_FIGARCH
Date,,,,,,,,
2012-07-13,0.017995,-0.005506,0.013420,0.013619,0.011274,0.000138,0.010527,0.013827
2012-07-20,0.011511,0.001549,0.013506,0.012398,0.010971,12.925793,0.011058,0.013577
2012-07-27,0.010579,0.004324,0.013579,0.011774,0.010235,12.533323,0.011510,0.013571
2012-08-03,0.010766,0.016962,0.013642,0.011488,0.009776,12.582447,0.011894,0.013826
2012-08-10,0.008131,0.003615,0.013696,0.011359,0.009491,12.199307,0.012221,0.013570
...,...,...,...,...,...,...,...,...
2014-05-23,0.012036,-0.000330,0.014022,0.011932,0.012704,13.583726,0.014089,0.013594
2014-05-30,0.006416,0.012000,0.014022,0.011559,0.011313,13.328481,0.014089,0.013672
2014-06-06,0.006691,0.012050,0.014022,0.011391,0.010448,13.085934,0.014089,0.013673
